In [13]:
import numpy as np
import os
import pandas as pd
from scipy.interpolate import RBFInterpolator
import matplotlib.pyplot as plt

In [14]:
# Folder path containing CSV files
folder_path = "data/csv"
out_file    = "results/losses/losses.csv"

# Ensure the output directory exists
os.makedirs(os.path.dirname(out_file), exist_ok=True)
write_header = not os.path.exists(out_file)

# Define value ranges
first_values = np.arange(0.3, 0.7, 0.05)  # 0.25 to 0.7 with step 0.05
# second_values = np.arange(3, 15, 1)  # 3 to 14
second_values = np.arange(3, 15, 1)  # 3 to 14

In [15]:
def numerical_gradient(f_interp, pts, h=1e-4):
    grads = []
    for i in range(2):
        offset = np.zeros_like(pts)
        offset[:, i] = h
        grad_i = (f_interp(pts + offset) - f_interp(pts - offset)) / (2 * h)
        grads.append(grad_i)
    return grads

def numerical_laplacian(f_interp, pts, h=1e-4):
    lap = np.zeros(len(pts))
    for i in range(2):
        offset = np.zeros_like(pts)
        f_plus = f_interp(pts + offset)
        f_minus = f_interp(pts - offset)
        f_center = f_interp(pts)
        lap += (f_plus - 2 * f_center + f_minus) / h**2
    return lap

In [ ]:
processed = set()
if os.path.exists(out_file):
    existing_df = pd.read_csv(out_file)
    processed = set(zip(existing_df['f'], existing_df['n']))

for first in first_values:
    for second in second_values:
        if (first, second) in processed:
            print(f"Skipping already processed f={first}, n={second}")
            continue
        
        first_str = f"{first:.2f}".rstrip("0").rstrip(".")
        file_name = f"W_{first_str}_{second}_1.csv"
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.exists(file_path):
            print(f"Processing {file_name}...")
            
            # Load CSV file
            df = pd.read_csv(file_path)
            
            # Extract input and output data
            points = df[['x', 'y']].values
            u = df['u'].values
            v = df['v'].values
            p = df['p'].values

            # Interpolators
            rbf_u = RBFInterpolator(points, u, kernel='thin_plate_spline')
            rbf_v = RBFInterpolator(points, v, kernel='thin_plate_spline')
            rbf_p = RBFInterpolator(points, p, kernel='thin_plate_spline')

            # Derivatives
            du_dx, du_dy = numerical_gradient(rbf_u, points)
            dv_dx, dv_dy = numerical_gradient(rbf_v, points)
            dp_dx, dp_dy = numerical_gradient(rbf_p, points)
            lap_u = numerical_laplacian(rbf_u, points)
            lap_v = numerical_laplacian(rbf_v, points)

            # Residuals
            mu = 0.01
            cont_res = du_dx + dv_dy
            mom_x_res = u * du_dx + v * du_dy + dp_dx - mu * lap_u
            mom_y_res = u * dv_dx + v * dv_dy + dp_dy - mu * lap_v

            # Plotting
            fig, axs = plt.subplots(1, 3, figsize=(18, 5))

            sc1 = axs[0].scatter(points[:, 0], points[:, 1], c=cont_res, cmap='coolwarm')
            axs[0].set_title('Continuity Residual')
            plt.colorbar(sc1, ax=axs[0])

            sc2 = axs[1].scatter(points[:, 0], points[:, 1], c=mom_x_res, cmap='coolwarm')
            axs[1].set_title('X-Momentum Residual')
            plt.colorbar(sc2, ax=axs[1])

            sc3 = axs[2].scatter(points[:, 0], points[:, 1], c=mom_y_res, cmap='coolwarm')
            axs[2].set_title('Y-Momentum Residual')
            plt.colorbar(sc3, ax=axs[2])

            for ax in axs:
                ax.set_xlabel('x')
                ax.set_ylabel('y')
                ax.set_aspect('equal')

            plt.tight_layout()
            plt.savefig(f"results/losses/residuals_{first_str}_{second}.png")

            # Mean absolute residuals
            avg_continuity = np.mean(np.abs(cont_res))
            avg_momentum_x = np.mean(np.abs(mom_x_res))
            avg_momentum_y = np.mean(np.abs(mom_y_res))

            row = {
                'f': first,
                'n': second,
                'continuity': avg_continuity,
                'momentum_x': avg_momentum_x,
                'momentum_y': avg_momentum_y
            }
            
            pd.DataFrame([row]).to_csv(
                out_file,
                mode='a',
                header=write_header,
                index=False
            )
            write_header = False  # after first write, never write header again

Skipping already processed f=0.3, n=3
Skipping already processed f=0.3, n=4
Skipping already processed f=0.3, n=5
Skipping already processed f=0.3, n=6
Skipping already processed f=0.3, n=7
Skipping already processed f=0.3, n=8
Skipping already processed f=0.3, n=9
Skipping already processed f=0.3, n=10
Skipping already processed f=0.3, n=11
Skipping already processed f=0.3, n=12
Skipping already processed f=0.3, n=13
Skipping already processed f=0.3, n=14
Skipping already processed f=0.35, n=3
Skipping already processed f=0.35, n=4
Skipping already processed f=0.35, n=5
Skipping already processed f=0.35, n=6
Skipping already processed f=0.35, n=7
Skipping already processed f=0.35, n=8
Processing W_0.35_9_1.csv...
